## 0. Preparación del notebook e inicialización del cliente de OpenAI API

In [1]:
import os
import pandas as pd
import json
import textwrap
from datetime import datetime
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv("../../../../../../../apis/.env")
api_key = os.getenv("OPENAI_API_KEY")
unmasked_chars = 8
masked_key = api_key[:unmasked_chars] + '*' * (len(api_key) - unmasked_chars*2) + api_key[-unmasked_chars:]
print(f"API key: {masked_key}")
client = OpenAI(api_key=api_key)
print("Cliente inicializado como",client)

API key: sk-proj-****************************************************************************************************************************************************-amA_5sA
Cliente inicializado como <openai.OpenAI object at 0x00000248A5AF9510>


## 1. Zero-shot named entity recognition

Empezamos con un caso sencillo extrayendo un texto del CV de ejemplo y sin especificar esquema para el diccionario de datos json:

In [2]:
text = "Vendedor/a de puesto de mercado - Mercadona"
# System prompt para reconocimiento de entidades nombradas (NER) de nombres de compañías y títulos de puestos de trabajo
ner_pre_prompt = (
  "Eres un procesador de currículos vitae que extrae nombres de "
  "compañías/empresas y títulos de puestos de trabajo. Usa formato json en la salida "
  'con las claves "empresa" y "puesto".'
)

response = client.chat.completions.create(
      model="gpt-4o-mini",
      response_format={"type": "json_object"}, # De momento no facilitamos esquema. Lo probaremos más adelante.
      messages=[
        {"role": "system", "content": ner_pre_prompt},
        {"role": "user", "content": text}
      ]
    )
generated_content = response.choices[0].message.content
print(generated_content)

{
  "empresa": "Mercadona",
  "puesto": "Vendedor/a de puesto de mercado"
}


Ejemplo de reconocimiento de entidades nombradas en un currículo completo. Hemos utilizado un CV de ejemplo no incluido en el repositorio. Para ejecutar el siguiente bloque, es necesario facilitar una ruta válida a un currículo:

In [3]:
cv_sample_path = '../../ejemplos_cvs/cv_sample.txt' # Ruta al fichero de texto con un currículo de ejemplo 
with open(cv_sample_path, 'r') as file:
    cv_text = file.read()
print(cv_text[:150],"...")

Candidato: Mohamed van der Poel Mendieta
Último Puesto Comercial de automoviles
Última formación reglada FP 1 / Técnico medio
3
Idioma EspañolInglésFr ...


Inferencia de entidades nombradas "empresa" y "puesto" con un modelo de OpenAI (elegimos gpt-4o-mini para reducir los costes y dado que esto sólo es una sencilla prueba de concepto)

In [4]:
response = client.chat.completions.create(
      model="gpt-4o-mini",
      response_format={"type": "json_object"},
      messages=[
        {"role": "system", "content": ner_pre_prompt},
        {"role": "user", "content": cv_text}
      ]
    )
generated_content = response.choices[0].message.content
print(generated_content)

{
  "experiencia": [
    {
      "empresa": "Autónomo",
      "puesto": "Comercial de automoviles"
    },
    {
      "empresa": "Mercadona",
      "puesto": "Vendedor/a de puesto de mercado"
    },
    {
      "empresa": "AGRISOLUTIONS",
      "puesto": "AUXILIAR DE MANTENIMIENTO INDUSTRIAL"
    },
    {
      "empresa": "GASTROTEKA ORDIZIA 1990",
      "puesto": "Camarero/a de barra"
    },
    {
      "empresa": "ZEREGUIN ZERBITZUAK",
      "puesto": "limpieza industrial"
    },
    {
      "empresa": "Bellota Herramientas",
      "puesto": "Personal de mantenimiento"
    }
  ]
}


### Procesamiento de fechas

Vamos a intentar extraer también las fechas para cada puesto de trabajo. Para ello, añadiremos algunas indicaciones adicionales en relación a los posibles formatos de entrada y al formato de salida. En cuanto a las entradas, asumimos que cada CV puede tener formatos muy distintos para esta información. Para las salidas, queremos un formato que nos facilite posteriormente realizar cálculos con fechas como la duración total, antigüedad con respecto a fecha actual, etc.

In [5]:
explicacion_fechas = (
    'Para el período, contempla cualquier formato de fecha o rango de fechas incluido en el texto. '
    'Un ejemplo de formato de fecha en la entrada es "Octubre 2023 / Marzo 2024". Otros ejemplos de '
    'formatos de fecha son "10/2023 - 03/2024", "Oct 2023 - Mar 2024", etc. '
    'El contenido para la clave "período" debe ser un string con dos elementos en formato YYYYMM '
    'separados por un guion, por ejemplo "202310-202403", o uno en caso de no identificarse fecha de fin.'
    )

ner_pre_prompt = (
  'Eres un procesador de currículos vitae que extrae títulos de puestos de trabajo, '
  'nombres de la empresa, y períodos de los mismos. Usa formato json en la salida '
  f'con las claves "empresa", "puesto" y "periodo". {explicacion_fechas}'
)
wrapped_ner_pre_prompt = textwrap.fill(ner_pre_prompt, width=100)
print(wrapped_ner_pre_prompt)

Eres un procesador de currículos vitae que extrae títulos de puestos de trabajo, nombres de la
empresa, y períodos de los mismos. Usa formato json en la salida con las claves "empresa", "puesto"
y "periodo". Para el período, contempla cualquier formato de fecha o rango de fechas incluido en el
texto. Un ejemplo de formato de fecha en la entrada es "Octubre 2023 / Marzo 2024". Otros ejemplos
de formatos de fecha son "10/2023 - 03/2024", "Oct 2023 - Mar 2024", etc. El contenido para la clave
"período" debe ser un string con dos elementos en formato YYYYMM separados por un guion, por ejemplo
"202310-202403", o uno en caso de no identificarse fecha de fin.


In [6]:
response = client.chat.completions.create(
      model="gpt-4o-mini",
      response_format={"type": "json_object"},
      messages=[
        {"role": "system", "content": ner_pre_prompt},
        {"role": "user", "content": cv_text}
      ]
    )
generated_content = response.choices[0].message.content
print(generated_content)

{
    "experiencia": [
        {
            "empresa": "Autónomo",
            "puesto": "Comercial de automoviles",
            "periodo": "202401-202402"
        },
        {
            "empresa": "Mercadona",
            "puesto": "Vendedor/a de puesto de mercado",
            "periodo": "202310-202403"
        },
        {
            "empresa": "AGRISOLUTIONS",
            "puesto": "AUXILIAR DE MANTENIMIENTO INDUSTRIAL",
            "periodo": "202001-202401"
        },
        {
            "empresa": "GASTROTEKA ORDIZIA 1990",
            "puesto": "Camarero/a de barra",
            "periodo": "202303-202309"
        },
        {
            "empresa": "ZEREGUIN ZERBITZUAK",
            "puesto": "limpieza industrial",
            "periodo": "202012-202305"
        },
        {
            "empresa": "Bellota Herramientas",
            "puesto": "Personal de mantenimiento",
            "periodo": "202005-202011"
        }
    ]
}


In [7]:
# Convertimos el texto en un objeto JSON
json_object = json.loads(generated_content)
# Convertimos a Pandas dataframe para realizar operaciones
# Aún no hemos especificado el esquema completo (a veces puede ser que el modelo nos dé "experiencias" en lugar de "experiencia")
df = pd.DataFrame(json_object["experiencia"]) 
display(df)

,empresa,puesto,periodo
0,Autónomo,Comercial de automoviles,202401-202402
1,Mercadona,Vendedor/a de puesto de mercado,202310-202403
2,AGRISOLUTIONS,AUXILIAR DE MANTENIMIENTO INDUSTRIAL,202001-202401
3,GASTROTEKA ORDIZIA 1990,Camarero/a de barra,202303-202309
4,ZEREGUIN ZERBITZUAK,limpieza industrial,202012-202305
5,Bellota Herramientas,Personal de mantenimiento,202005-202011


Antes de desarrollar el código para la extracción y tratamiento de fechas, vamos a comprobar si el modelo es capaz de procesar correctamente un puesto sin fecha de fin en el período. Vamos a eliminar la fecha de fin en el puesto "comercial de automóviles" y guardarlo en '../../ejemplos_cvs/cv_sample_2.txt' (esta ruta no está incluida en el repositorio)

In [8]:
cv_sample_2_path = '../../ejemplos_cvs/cv_sample_2.txt'
with open(cv_sample_2_path, 'r') as file:
    cv_text_2 = file.read()
print(f"### Ejemplo original ###\n...\n{cv_text[301:386]}\n...")
print(f"\n### Ejemplo modificado ###\n...\n{cv_text_2[301:371]}\n...")

### Ejemplo original ###
...
Sexo Hombre
Experiencia
Enero 2024 / Febrero 2024
Comercial de automoviles - Autónomo
...

### Ejemplo modificado ###
...
Sexo Hombre
Experiencia
Enero 2024
Comercial de automoviles - Autónomo
...


Volvemos a pedir la inferencia con el CV modificado:

In [9]:
response = client.chat.completions.create(
      model="gpt-4o-mini",
      response_format={"type": "json_object"},
      messages=[
        {"role": "system", "content": ner_pre_prompt},
        {"role": "user", "content": cv_text_2} # Sin fecha de fin en la última experiencia
      ]
    )
generated_content = response.choices[0].message.content
print(generated_content)

{
  "experiencia": [
    {
      "empresa": "Autónomo",
      "puesto": "Comercial de automoviles",
      "periodo": "202401"
    },
    {
      "empresa": "Mercadona",
      "puesto": "Vendedor/a de puesto de mercado",
      "periodo": "202310-202404"
    },
    {
      "empresa": "AGRISOLUTIONS",
      "puesto": "AUXILIAR DE MANTENIMIENTO INDUSTRIAL",
      "periodo": "202001-202401"
    },
    {
      "empresa": "GASTROTEKA ORDIZIA 1990",
      "puesto": "Camarero/a de barra",
      "periodo": "202303-202309"
    },
    {
      "empresa": "ZEREGUIN ZERBITZUAK",
      "puesto": "limpieza industrial",
      "periodo": "202012-202305"
    },
    {
      "empresa": "Bellota Herramientas",
      "puesto": "Personal de mantenimiento",
      "periodo": "202005-202011"
    }
  ]
}


Vemos que el modelo gpt-4o-mini parece suficientemente solvente procesando e interpretando datos no estructurados como fechas. En un caso de uso real en el que dispongamos de muchos ficheros de entrada, podríamos entrenar un modelo de "named entity recognition" más sofisticado para asegurar mayor precisión. 

<br> A continuación, procedemos a tratar las fechas para definir un parámetro de duración del puesto de trabajo: 

In [10]:
# Convertimos el texto en un objeto JSON
json_object = json.loads(generated_content)
# Convertimos a Pandas dataframe para realizar operaciones
df_experiencia = pd.DataFrame(json_object["experiencia"])
display(df_experiencia)

,empresa,puesto,periodo
0,Autónomo,Comercial de automoviles,202401
1,Mercadona,Vendedor/a de puesto de mercado,202310-202404
2,AGRISOLUTIONS,AUXILIAR DE MANTENIMIENTO INDUSTRIAL,202001-202401
3,GASTROTEKA ORDIZIA 1990,Camarero/a de barra,202303-202309
4,ZEREGUIN ZERBITZUAK,limpieza industrial,202012-202305
5,Bellota Herramientas,Personal de mantenimiento,202005-202011


In [11]:
# Función para procesar el período
def split_periodo(periodo):
    dates = periodo.split('-')
    start_date = datetime.strptime(dates[0], "%Y%m")
    if len(dates) > 1:
        end_date = datetime.strptime(dates[1], "%Y%m")
    else:
        end_date = datetime.now()
    return start_date, end_date

df_experiencia[['fec_inicio', 'fec_final']] = df_experiencia['periodo'].apply(lambda x: pd.Series(split_periodo(x)))

# Formateamos las fechas para mostrar mes, año, y el primer día del mes (dado que el día es irrelevante y no se suele especificar)
df_experiencia['fec_inicio'] = df_experiencia['fec_inicio'].dt.date
df_experiencia['fec_final'] = df_experiencia['fec_final'].dt.date

# Añadimos una columna con la duración en meses
df_experiencia['duracion'] = df_experiencia.apply(
    lambda row: (row['fec_final'].year - row['fec_inicio'].year) * 12 + 
                row['fec_final'].month - row['fec_inicio'].month, 
    axis=1
)

display(df_experiencia)

,empresa,puesto,periodo,fec_inicio,fec_final,duracion
0,Autónomo,Comercial de automoviles,202401,2024-01-01,2024-12-08,11
1,Mercadona,Vendedor/a de puesto de mercado,202310-202404,2023-10-01,2024-04-01,6
2,AGRISOLUTIONS,AUXILIAR DE MANTENIMIENTO INDUSTRIAL,202001-202401,2020-01-01,2024-01-01,48
3,GASTROTEKA ORDIZIA 1990,Camarero/a de barra,202303-202309,2023-03-01,2023-09-01,6
4,ZEREGUIN ZERBITZUAK,limpieza industrial,202012-202305,2020-12-01,2023-05-01,29
5,Bellota Herramientas,Personal de mantenimiento,202005-202011,2020-05-01,2020-11-01,6


In [ ]:
df_experiencia.to_pickle('../pkl/df_experiencia.pkl') # Guardamos pickle para usarlo en el siguiente notebook

## 2. NER con sequema para "structured output" y llamada a función

Explicar lo que necesitamos en el prompt y poner "json_object" en "response_format" parece más suficiente para obtener buenos resultados la mayoría de las veces. Sin embargo, nos podemos encontrar con problemas como, por ejemplo, que el modelo no siempre nos dé la misma palabra como clave de primer nivel (a veces puede poner "experiencia", otras veces "experiencias", "roles"...). Se podría intentar explicar esto con lenguaje natural en el prompt, pero es más sencillo definir un esquema y definirlo como función.

Sin embargo, para asegurar que el modelo siempre responda con un formato consistente, podemos definir un esquema:

In [21]:
explicacion_fechas = (
    'Para el período, contempla cualquier formato de fecha o rango de fechas incluido en el texto. '
    'Un ejemplo de formato de fecha en la entrada es "Octubre 2023 / Marzo 2024". '
    'El contenido para la clave "período" debe ser un string con dos elementos en formato YYYYMM '
    'separados por un guion, por ejemplo "202310-202403", o uno en caso de no identificarse fecha de fin.'
    )

ner_pre_prompt = (
  'Eres un procesador de currículos vitae que extrae títulos de puestos de trabajo, '
  'nombres de la empresa, y períodos de los mismos. Usa formato json en la salida '
  f'con las claves "empresa", "puesto" y "periodo". {explicacion_fechas}'
)

# Guardamos el prompt para el reconocimiento de entidades nombradas en un archivo de texto
with open('../prompts/ner_pre_prompt.txt', 'w', encoding='utf-8') as file:
    file.write(ner_pre_prompt)

wrapped_ner_pre_prompt = textwrap.fill(ner_pre_prompt, width=100)
print(wrapped_ner_pre_prompt)
cv_sample_2_path = '../../ejemplos_cvs/cv_sample_2.txt'
with open(cv_sample_2_path, 'r') as file:
    cv_text_2 = file.read()

Eres un procesador de currículos vitae que extrae títulos de puestos de trabajo, nombres de la
empresa, y períodos de los mismos. Usa formato json en la salida con las claves "empresa", "puesto"
y "periodo". Para el período, contempla cualquier formato de fecha o rango de fechas incluido en el
texto. Un ejemplo de formato de fecha en la entrada es "Octubre 2023 / Marzo 2024". El contenido
para la clave "período" debe ser un string con dos elementos en formato YYYYMM separados por un
guion, por ejemplo "202310-202403", o uno en caso de no identificarse fecha de fin.


In [14]:
# Definimos el esquema en formato JSON
schema = {
    "type": "object",
    "properties": {
        "records": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "empresa": {"type": "string"},
                    "puesto": {"type": "string"},
                    "periodo": {
                        "type": "string",
                        "description": "Formato 'YYYYMM-YYYYMM' o simplemente 'YYYYMM' si no aparece fecha de fin."
                    }
                },
                "required": ["empresa", "puesto", "periodo"]
            }
        }
    },
    "required": ["records"]
}

# Llamamos a la API, incluyendo el esquema deseado en el parámetro 'functions'
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": ner_pre_prompt},
        {"role": "user", "content": cv_text}
    ],
    functions=[
        {
            "name": "extraer_datos_cv",
            "description": "Extrae tabla con títulos de puesto de trabajo, nombres de empresa y períodos de un CV.",
            "parameters": schema
        }
    ],
    function_call="auto"
)

# Extraemos de la respuesta sólo los datos de la función
if response.choices[0].message.function_call:
    function_call = response.choices[0].message.function_call
    structured_output = json.loads(function_call.arguments)
    print("Datos estructurados:\n", json.dumps(structured_output, indent=4, ensure_ascii=False))
else:
    print("No se han podido extraer datos estructurados.")

Datos estructurados:
 {
    "records": [
        {
            "empresa": "Autónomo",
            "puesto": "Comercial de automoviles",
            "periodo": "202401-202402"
        },
        {
            "empresa": "Mercadona",
            "puesto": "Vendedor/a de puesto de mercado",
            "periodo": "202310-202403"
        },
        {
            "empresa": "AGRISOLUTIONS",
            "puesto": "AUXILIAR DE MANTENIMIENTO INDUSTRIAL",
            "periodo": "202001-202401"
        },
        {
            "empresa": "GASTROTEKA ORDIZIA 1990",
            "puesto": "Camarero/a de barra",
            "periodo": "202303-202309"
        },
        {
            "empresa": "ZEREGUIN ZERBITZUAK",
            "puesto": "limpieza industrial",
            "periodo": "202012-202305"
        },
        {
            "empresa": "Bellota Herramientas",
            "puesto": "Personal de mantenimiento",
            "periodo": "202005-202011"
        }
    ]
}


## 3. NER con esquema en fichero .JSON

Para desarrollar el código ejecutable más adelante, vamos a utilizar un fichero .json externo con el esquema, lo que facilita el control de versiones y simplifica el código:

In [17]:
# Cargamos el esquema:
with open('../json/ner_schema.json', 'r', encoding='utf-8') as schema_file:
    schema = json.load(schema_file)

# Cargamos el CV:
cv_sample_path = '../../ejemplos_cvs/cv_sample.txt' # Ruta al fichero de texto con un currículo de ejemplo
with open(cv_sample_path, 'r') as file:
    cv_text = file.read()

def extraer_datos_cv(pre_prompt, schema, cv, temperature=0.5):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=temperature,
        messages=[
            {"role": "system", "content": pre_prompt},
            {"role": "user", "content": cv}
        ],
        functions=[
            {
                "name": "extraer_datos_cv",
                "description": "Extrae tabla con títulos de puesto de trabajo, nombres de empresa y períodos de un CV.",
                "parameters": schema
            }
        ],
        function_call="auto"
    )

    if response.choices[0].message.function_call:
        function_call = response.choices[0].message.function_call
        structured_output = json.loads(function_call.arguments)
        if structured_output.get("experiencia"):
            return structured_output
        else:
            return {"error": f"No se han podido extraer datos estructurados: {response.choices[0].message.content}"}
    else:
        return {"error": f"No se han podido extraer datos estructurados: {response.choices[0].message.content}"}
    
datos_estructurados_cv = extraer_datos_cv(ner_pre_prompt, schema, cv_text)
print("Datos estructurados:\n", json.dumps(datos_estructurados_cv, indent=4, ensure_ascii=False))

Datos estructurados:
 {
    "experiencia": [
        {
            "empresa": "Autónomo",
            "puesto": "Comercial de automoviles",
            "periodo": "202401-202402"
        },
        {
            "empresa": "Mercadona",
            "puesto": "Vendedor/a de puesto de mercado",
            "periodo": "202310-202403"
        },
        {
            "empresa": "AGRISOLUTIONS",
            "puesto": "AUXILIAR DE MANTENIMIENTO INDUSTRIAL",
            "periodo": "202001-202401"
        },
        {
            "empresa": "GASTROTEKA ORDIZIA 1990",
            "puesto": "Camarero/a de barra",
            "periodo": "202303-202309"
        },
        {
            "empresa": "ZEREGUIN ZERBITZUAK",
            "puesto": "limpieza industrial",
            "periodo": "202012-202305"
        },
        {
            "empresa": "Bellota Herramientas",
            "puesto": "Personal de mantenimiento",
            "periodo": "202005-202011"
        }
    ]
}


## Pruebas adicionales

En las siguientes pruebas, experimentamos con modificaciones del parámetro de temperatura en casos extremos de textos atípicos. El objetivo principal es asegurar que el agente extraiga toda la información válida posible pero, a la vez, evite "alucinar" cuando reciba datos confusos. Un parámetro muy alto de temperatura puede producir algunas alucinaciones en casos muy excepcionales, por lo que usaremos un parámetro muy "conservador". En cualquier caso, las pruebas son suficientes para estar muy "cómodos" con la efectividad del modelo gpt-4o-mini en esta tarea: tiene un rendimiento muy sólido.

Currículum "minimalista":

In [18]:
cv_text_mini = "Soy un vendedor de puesto de mercado en Mercadona. Antes trabajé como camarero en un bar de tapas."
datos_estructurados_cv_mini = extraer_datos_cv(ner_pre_prompt, schema, cv_text_mini, temperature=0.1)
print("Datos estructurados:\n", json.dumps(datos_estructurados_cv_mini, indent=4, ensure_ascii=False))

Datos estructurados:
 {
    "experiencia": [
        {
            "empresa": "Mercadona",
            "puesto": "Vendedor",
            "periodo": ""
        },
        {
            "empresa": "Bar de tapas",
            "puesto": "Camarero",
            "periodo": ""
        }
    ]
}


Texto inválido:

In [19]:
cv_text_hal = (
    "El rápido zorro marrón salta sobre el perezoso perro. El perro ladra al zorro. "
    "Los dos animales se miran fijamente. Es una escena común en el bosque. Me gusta el bosque."
)

datos_estructurados_cv_hal = extraer_datos_cv(ner_pre_prompt, schema, cv_text_hal, temperature=0.1)
print("Datos estructurados:\n", json.dumps(datos_estructurados_cv_hal, indent=4, ensure_ascii=False))

Datos estructurados:
 {
    "error": "No se han podido extraer datos estructurados: None"
}


In [20]:
cv_text_hal = (
    "El rápido zorro marrón salta sobre el perezoso perro. El perro ladra al zorro. "
    "Los dos animales se miran fijamente. Es una escena común en el bosque. Me gusta el bosque."
)

datos_estructurados_cv_hal = extraer_datos_cv(ner_pre_prompt, schema, cv_text_hal, temperature=2)
print("Datos estructurados:\n", json.dumps(datos_estructurados_cv_hal, indent=4, ensure_ascii=False))

Datos estructurados:
 {
    "error": "No se han podido extraer datos estructurados: None"
}
